# LandScapeMini
End‑to‑end pipeline: **download → train EfficientNet‑B2 → evaluate → quick inference → Gradio app**.



In [1]:
from google.colab import drive
drive.mount('<YOUR_PATH_HERE>/drive')

Mounted at /content/drive


> **Tip:** run all cells top-to-bottom. If you get a CUDA OOM, lower the batch size.

In [2]:
!pip -q install --upgrade pip
!pip -q install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.5 MB/s eta 0:00:00


In [3]:
import os, zipfile, urllib.request, random, time, shutil
from pathlib import Path

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.models import efficientnet_b2, EfficientNet_B2_Weights

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("Torch version:", torch.__version__)
print("Device:", DEVICE)

Torch version: 2.8.0+cu126
Device: cuda


In [4]:
# Download and extract
import shutil
from pathlib import Path
import zipfile

from google.colab import drive
drive.mount('<YOUR_PATH_HERE>/drive')

MYDRIVE = Path("<YOUR_PATH_HERE>/<YOUR_PATH_HERE>/")
ZIP_PATH = MYDRIVE / "archive.zip"
DATASET_DIR = MYDRIVE / "dataset"
TRAIN_DIR = DATASET_DIR / "train"
TEST_DIR  = DATASET_DIR / "test"

def extract_and_reorganize(zip_path, dataset_dir):
    if dataset_dir.exists() and any(dataset_dir.iterdir()):
        print("[INFO] Dataset already exists at:", dataset_dir)
    else:
        print(f"[INFO] Extracting {zip_path.name} ...")
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(dataset_dir)
        print("[INFO] Extracted to:", dataset_dir)

    seg_train_inner = next(dataset_dir.glob("seg_train/seg_train"), None)
    seg_test_inner  = next(dataset_dir.glob("seg_test/seg_test"), None)

    if seg_train_inner and not TRAIN_DIR.exists():
        shutil.move(str(seg_train_inner), str(TRAIN_DIR))
        print("[INFO] Moved seg_train ->", TRAIN_DIR)

    if seg_test_inner and not TEST_DIR.exists():
        shutil.move(str(seg_test_inner), str(TEST_DIR))
        print("[INFO] Moved seg_test ->", TEST_DIR)

    print("[INFO] Final structure ready.")
    return TRAIN_DIR, TEST_DIR

train_dir, test_dir = extract_and_reorganize(ZIP_PATH, DATASET_DIR)

print("Train dir:", train_dir)
print("Test dir :", test_dir)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[INFO] Extracting archive.zip ...
[INFO] Extracted to: /content/drive/MyDrive/dataset
[INFO] Moved seg_train -> /content/drive/MyDrive/dataset/train
[INFO] Moved seg_test -> /content/drive/MyDrive/dataset/test
[INFO] Final structure ready.
Train dir: /content/drive/MyDrive/dataset/train
Test dir : /content/drive/MyDrive/dataset/test


In [5]:
MYDRIVE = Path("<YOUR_PATH_HERE>/<YOUR_PATH_HERE>/")
ZIP_PATH = MYDRIVE / "archive.zip"
DATASET_DIR = MYDRIVE / "dataset"
test_dir  = DATASET_DIR / "test"

In [6]:
# DataLoaders
BATCH_SIZE = 32
NUM_WORKERS = 2

weights = EfficientNet_B2_Weights.DEFAULT
transform = weights.transforms()

train_ds = datasets.ImageFolder(train_dir, transform=transform)
test_ds  = datasets.ImageFolder(test_dir,  transform=transform)
class_names = test_ds.classes
print("Classes:", class_names)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

Classes: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [7]:
# cache weights in the dataset folder
import os, torch
from pathlib import Path


os.environ["TORCH_HOME"] = str(DATASET_DIR)
torch.hub.set_dir(str(DATASET_DIR))
print("[INFO] Torch cache dir:", torch.hub.get_dir())


[INFO] Torch cache dir: /content/drive/MyDrive/dataset


In [8]:
# EfficientNet-B2
num_classes = len(class_names)
model = efficientnet_b2(weights=weights)

for p in model.features.parameters():
    p.requires_grad = False

# Replace classifier head to match 6 classes
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, num_classes)

model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /content/drive/MyDrive/dataset/checkpoints/efficientnet_b2_rwightman-c35c1473.pth


100%|██████████| 35.2M/35.2M [00:00<00:00, 102MB/s]


In [9]:
# Training + evaluation
EPOCHS = 5

def run_epoch(model, loader, train=True):
    if train:
        model.train()
    else:
        model.eval()
    total_loss, correct, total = 0.0, 0, 0
    if train:
        for X, y in loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad(set_to_none=True)
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * X.size(0)
            pred = logits.argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    else:
        with torch.inference_mode():
            for X, y in loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss = criterion(logits, y)
                total_loss += loss.item() * X.size(0)
                pred = logits.argmax(1)
                correct += (pred == y).sum().item()
                total += y.size(0)
    return (total_loss / total), (correct / total)

for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_epoch(model, train_loader, train=True)
    te_loss, te_acc = run_epoch(model, test_loader, train=False)
    print(f"Epoch {epoch:02d}/{EPOCHS} | train_acc={tr_acc:.3f} | test_acc={te_acc:.3f}")

Epoch 01/5 | train_acc=0.825 | test_acc=0.890
Epoch 02/5 | train_acc=0.868 | test_acc=0.889
Epoch 03/5 | train_acc=0.876 | test_acc=0.898
Epoch 04/5 | train_acc=0.882 | test_acc=0.900
Epoch 05/5 | train_acc=0.885 | test_acc=0.896


In [10]:
WEIGHTS_PATH = Path("<YOUR_PATH_HERE>/<YOUR_PATH_HERE>//dataset/landscape_mini_effnetb2.pth")
torch.save(model.state_dict(), WEIGHTS_PATH)
print("Saved weights to:", WEIGHTS_PATH)

Saved weights to: /content/drive/MyDrive/dataset/landscape_mini_effnetb2.pth


In [13]:
# Quick inference preview
import itertools
import torch.nn.functional as F

def predict_paths(paths):
    model.eval()
    results = []
    with torch.inference_mode():
        for p in paths:
            from PIL import Image
            img = Image.open(p).convert("RGB")
            x = transform(img).unsqueeze(0).to(DEVICE)
            logits = model(x)
            probs = F.softmax(logits, dim=1)[0].cpu().tolist()
            top_class = class_names[int(torch.argmax(logits, dim=1).cpu())]
            results.append((str(p), top_class, {class_names[i]: round(probs[i], 4) for i in range(len(class_names))}))
    return results

# grab three images from test set
sample_paths = []
for cls_dir in (test_dir / class_names[0], test_dir / class_names[1], test_dir / class_names[2]):
    for p in cls_dir.iterdir():
        if p.suffix.lower() in [".jpg", ".jpeg", ".png"]:
            sample_paths.append(p)
            break

preview = predict_paths(sample_paths)
for path, top, probdict in preview:
    print(f"{path} -> {top} | {probdict}")

/content/drive/MyDrive/dataset/test/buildings/20057.jpg -> buildings | {'buildings': 0.9947, 'forest': 0.0, 'glacier': 0.0001, 'mountain': 0.0002, 'sea': 0.0001, 'street': 0.005}
/content/drive/MyDrive/dataset/test/forest/20056.jpg -> forest | {'buildings': 0.0, 'forest': 0.9999, 'glacier': 0.0, 'mountain': 0.0, 'sea': 0.0, 'street': 0.0001}
/content/drive/MyDrive/dataset/test/glacier/20059.jpg -> glacier | {'buildings': 0.0227, 'forest': 0.0033, 'glacier': 0.8674, 'mountain': 0.0329, 'sea': 0.0454, 'street': 0.0283}


In [14]:
# Prepare small examples folder for Gradio
import os
EX_DIR = Path("<YOUR_PATH_HERE>/<YOUR_PATH_HERE>//dataset/examples")
if EX_DIR.exists():
    shutil.rmtree(EX_DIR)
EX_DIR.mkdir(parents=True, exist_ok=True)

# copy the same 3 sample images
for p in list((test_dir / class_names[0]).glob("*"))[:1] +          list((test_dir / class_names[1]).glob("*"))[:1] +          list((test_dir / class_names[2]).glob("*"))[:1]:
    if p.suffix.lower() in [".jpg", ".jpeg", ".png"]:
        shutil.copy2(p, EX_DIR / p.name)

print("Examples:", [str(p) for p in EX_DIR.iterdir()])

Examples: ['/content/drive/MyDrive/dataset/examples/20057.jpg', '/content/drive/MyDrive/dataset/examples/20056.jpg', '/content/drive/MyDrive/dataset/examples/20059.jpg']


In [15]:
# Gradio
import gradio as gr
import torch.nn.functional as F
from PIL import Image

def predict_gradio(img):
    model.eval()
    with torch.inference_mode():
        x = transform(img).unsqueeze(0).to(DEVICE)
        logits = model(x)
        probs = F.softmax(logits, dim=1)[0].cpu().tolist()
    return {class_names[i]: float(probs[i]) for i in range(len(class_names))}

examples = [[str(p)] for p in EX_DIR.iterdir() if p.suffix.lower() in [".jpg", ".jpeg", ".png"]]

demo = gr.Interface(
    fn=predict_gradio,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=6),
    title="Landscape Mini (EfficientNet‑B2)",
    description="Upload a landscape image to classify.",
    examples=examples
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://75199853f2412a1ba1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
